In [1]:
from pymongo import MongoClient

In [2]:
import numpy as np

In [3]:
import pandas as pd

In [4]:
from collections import defaultdict

In [5]:
mongo = MongoClient()

In [6]:
db = mongo.declarator

In [7]:
db.collection_names()

['declarations']

In [87]:
from person_to_features import PersonToFeatures

In [88]:
p2f = PersonToFeatures()

In [96]:
people = db.declarations.aggregate([{'$project': {'pid': '$main.person.id'}}, {'$sort': {'pid': 1}}])

In [97]:
all_ids = list(set(map(
    lambda x: x['pid'],
    people
)))

In [98]:
len(all_ids)

51909

In [110]:
data = []
found_people = []
regions = []

for i in all_ids:
    if i % 500 == 0:
        print(i)
    
    person = list(db.declarations.find({
        'main.person.id': i,
        'main.document_type.id': 1
    }))
    
    if len(person) > 1:
        person_regions = set(map(lambda x: x[0]['name'], p2f.get_possible_regions(person)))
        regions.append(person_regions)
        found_people.append(i)
        found_people.append((i, person[0]['main']['person']['name']))
        
        features = p2f.generate_timeline_features(person)
        data.append(features)

500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500
8000
8500
9000
9500
10000
10500
11000
11500
12000
12500
13000
13500
14000
14500
15000
15500
16000
16500
17000
17500
18000
18500
19000
19500
20000
20500
21000
21500
22000
22500
23000
23500
24000
24500
25000
25500
26000
26500
27000
27500
28000
28500
29000
29500
30000
30500
31000
31500
32000
32500
33000
33500
34000
34500
35000
35500
36000
36500
37000
37500
38000
38500
39000
39500
40000
40500
41000
41500
42000
42500
43000
43500
44000
44500
45000
45500
46000
46500
47000
47500
48000
48500
49000
49500
50000
50500
51000
51500
52000


In [112]:
all_features = pd.DataFrame(data)

In [114]:
all_features.to_csv('data/all_features.csv', index=None)

In [119]:
names = pd.DataFrame(list(map(list, filter(lambda x: isinstance(x, tuple), found_people))), columns=['id', 'name'])

In [120]:
names.to_csv('data/all_names.csv', index=None)

In [129]:
regions = list(map(
    lambda x: next(iter(x), ''),
    regions
))

In [130]:
all_features['region'] = regions

In [132]:
all_features.to_csv('data/all_features_with_region.csv', index=None)